In [1]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

In [2]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out


installations:

1. pip install synapseclient

In [3]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out


In [4]:
import os
import pandas as pd
import numpy as np

full_fragments_and_cell_type_labels = [("/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118181_syn52120036_ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz/ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz",
                                        "/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/labels/ENCSR618WVK/cell_types.tsv.gz")]
# full_fragments_and_cell_type_labels = [("/Users/eilaarich-landkof-stanford/Downloads/k562/k562_err9847049_rna_atac_common_filtered_fragments.tsv.gz",
#                                         "/Users/eilaarich-landkof-stanford/Downloads/k562/barcode_cell_types.tsv.gz")]



local_clusters_fld = os.path.join(os.getcwd(),"clusters")
os.makedirs(local_clusters_fld, exist_ok=True)
local_path_to_download = os.path.join(os.getcwd(),"fragment_files")

In [5]:
local_clusters_fld

'/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters'

In [6]:
local_path_to_download

'/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files'

In [7]:
def remove_file(filename):
  print("remove_file method: {}".format(filename))
  if os.path.exists(filename):
    os.remove(filename)

In [8]:
def save_clusters_mapping(fld,atac_dataset_id,list_to_save):
 # Create a DataFrame from the tuple list
    df = pd.DataFrame(list_to_save, columns=["ClusterId", "ManualAnnotationLabel"])

    # Specify the output TSV file path
    output_tsv_file = os.path.join(fld,atac_dataset_id,"{}_cluster_id_annotation_map.tsv".format(atac_dataset_id))
    print("sort_list_with_indices_and_save: output_tsv_file is {}".format(output_tsv_file))
    # Write the DataFrame to a TSV file
    df.to_csv(output_tsv_file, sep="\t", index=False)

In [9]:
# IMPORTNAT: we make sure to keep the same order if the cell_id and the cluster for the next execution
def sort_list_with_indices(input_list):
    # print("input_list is {}".format(input_list))
    sorted_list = sorted(input_list)
    # print("sorted_list is {}".format(sorted_list))
    indexed_sorted_list = [(i, item) for i, item in enumerate(sorted_list)]
    print("indexed_sorted_list is {}".format(indexed_sorted_list))

    
    return indexed_sorted_list

In [10]:

# Fragment - ENCSR139ATR
# metadata - atac_dataset column => ENCSR139ATR => all the cell_id :ENCSR369UFT-1_ENCSR139ATR-1_CCTGGTAAGCAAGATG.
# Look for them in cell_type_4 (ENCSR369UFT-1_ENCSR139ATR-1_CCTGGTAAGCAAGATG)

# Cell type annotation - 

In [11]:
%run tsv_files_utils_from_analysis.ipynb
from itertools import islice


for local_file_tuple in full_fragments_and_cell_type_labels:

    # Dictionary to store the output file handles with names
    output_handles = {}
    local_fragment_file = local_file_tuple[0]
    full_cell_types_annotation_file_path = local_file_tuple[1]
    print("!!!!!local_fragment_file is {}".format(local_fragment_file))
    file_atac_dataset_id = local_fragment_file.split("_")[-1].split(".")[0]
    print("$$$$$$file_atac_dataset_id is {}".format(file_atac_dataset_id))
 
    skip_rows=0
    df_cell_types_for_atac_dataset = read_tsv_gz_to_dataframe_skipping_empty_lines_and_rows(full_cell_types_annotation_file_path,skip_rows)
    print("df_cell_types_for_atac_dataset.head(2) is {}".format(df_cell_types_for_atac_dataset.head(2)))
    cell_type_id_names_for_atac_dataset = list(set(df_cell_types_for_atac_dataset['cell_type_id'].to_list()))
    # Remove the None values from the list
    cell_type_id_names_for_atac_dataset = [x for x in cell_type_id_names_for_atac_dataset if not pd.isna(x)]
    print("cell_type_id_names_for_atac_dataset is {}".format(cell_type_id_names_for_atac_dataset))
    sorted_idx_cell_type_id_names_for_atac_dataset = sort_list_with_indices(cell_type_id_names_for_atac_dataset)
    print("%%%%%%sorted_idx_cell_type_id_names_for_atac_dataset is {}".format(sorted_idx_cell_type_id_names_for_atac_dataset))
    print("^^^^^^^number of cell_type_id_names_for_atac_dataset is {}".format(len(sorted_idx_cell_type_id_names_for_atac_dataset)))


    cell_id_to_type_dict = dict((key, value) for key, value in zip(df_cell_types_for_atac_dataset['cell_id'], df_cell_types_for_atac_dataset['cell_type_id']) if str(key)!="nan")
    cell_id_to_type_dict = {key: value for key, value in cell_id_to_type_dict.items() if not isinstance(value, float) or not np.isnan(value)}    
    print("\n".join([f"Cell ID: {cell_id}, Cell Type ID: {cell_type_id}" for cell_id, cell_type_id in islice(cell_id_to_type_dict.items(), 5)]))
    
    # List of output text files with corresponding names
    # [("output1.txt", "file_1"), ("output2.txt", "file_2"), ("output3.txt", "file_3")]
    output_tagAlign_files_with_names = [(os.path.join(local_clusters_fld,file_atac_dataset_id,"{}_Cluster{}.tsv.tagAlign".format(file_atac_dataset_id,item[0])),
                                    item[1]) for item in sorted_idx_cell_type_id_names_for_atac_dataset]
    print("@@@@output_tagAlign_files_with_names is {}".format(output_tagAlign_files_with_names))

    # this will make sure that we will not run the same tagAlign twice.
    tagAlign_exists = [os.path.exists(output_tagAlign_file[0]) for output_tagAlign_file in output_tagAlign_files_with_names]
    # print("tagAlign_exists is {}".format(tagAlign_exists))
    if sum(tagAlign_exists) >0:
        print("output_tagAlign_files_with_names {} is at work or was already downloaded. continue".format(output_tagAlign_files_with_names))
        continue # either started by annother process or already was processed
    else:
        print("!!!output_tagAlign_files_with_names {}. open files".format(output_tagAlign_files_with_names))
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            # print("tag_file_path is {}".format(tag_file_path))
            # print("os.path.dirname(tag_file_path) is {}".format(os.path.dirname(tag_file_path)))
            os.makedirs(os.path.dirname(tag_file_path), exist_ok=True)
            output_handles[tag_file_cell_type_name] = open(tag_file_path, "w")

    print("save the cluster to annotation mapping")
    save_clusters_mapping(local_clusters_fld,file_atac_dataset_id,sorted_idx_cell_type_id_names_for_atac_dataset)
    
    # print("!!!output_tagAlign_files_with_names {}".format(output_tagAlign_files_with_names))
    print("open local_fragment_file {}".format(local_fragment_file))
    with gzip.open(local_fragment_file, "rt") as infile:
        missing_bc = 0
        # Open the output files and store their handles in the list
        num_of_lines_written=0
        for line_number, line in enumerate(infile, start=1):
            # # debug
            # if line_number > 10000:
            #     continue 

            # here cases where bc_datasetId or datasetId_bc are being mixed between the fragments and the
            # cell type are being address. you can select the righ out_list for your experiment
            # print("\n\nline is {}".format(line))
            out_list = split_fragment_line_string(line)
            # print("\n\nout_list is {}".format(out_list))
            bc = out_list[-1]
            # print("bc is {}".format(bc))
#             Austin output: chrom, start, end, bc, rem = line.rstrip('\n').split('\t', 5)
            out_line_to_print = f"{out_list[0]}\t{out_list[1]}\t{out_list[2]}\t{bc}\t{out_list[-2]}\n"
#             chr1	10007	10175	ENCSR023FME#ENCSR023FME_GAAGGTTCAAAGTGTCAGTCAA	1
            num_of_lines_written +=1
            returnTagAlign = convert_fragment_line_to_tagAlign(out_line_to_print)
            # print(" returnTagAlign is {}".format(returnTagAlign))
            try:
                tag_file_cell_type_id = cell_id_to_type_dict[bc] #df_cell_types_for_atac_dataset.loc[df_cell_types_for_atac_dataset['cell_id'] == bc, 'cell_type_id'].iloc[0]
                # print("tag_file_cell_type_id is {}".format(tag_file_cell_type_id))
                output_handles[tag_file_cell_type_id].write(returnTagAlign)
            except:
                missing_bc+=1
                # print("df_cell_types_for_atac_dataset['cell_id'] is {}".format(df_cell_types_for_atac_dataset[df_cell_types_for_atac_dataset['cell_id'] == bc]))
                # print("didnt find bc {}".format(bc))
        
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            print("tag_file_path is {}".format(tag_file_path))
            output_handles[tag_file_cell_type_name].close()    
        print("finished clustering local_fragment_file {} by cell type. for types {}".format(local_fragment_file, cell_type_id_names_for_atac_dataset))
        print("total missing bc are {}".format(missing_bc))

!!!!!local_fragment_file is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118181_syn52120036_ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz/ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz
$$$$$$file_atac_dataset_id is ENCSR618WVK
tsv_gz_file is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/labels/ENCSR618WVK/cell_types.tsv.gz
df_cell_types_for_atac_dataset.head(2) is                                cell_id_original  rna_dataset  \
0  ENCSR369UFT-1_ENCSR139ATR-1_AAACAGCCAACACCTA  ENCSR369UFT   
1  ENCSR369UFT-1_ENCSR139ATR-1_AAACAGCCAAGCGATG  ENCSR369UFT   

        rna_barcode atac_dataset      atac_barcode  rna_umi_count  \
0  AAACAGCCAACACCTA  ENCSR139ATR  TGAGGAAACCCGCTGT          11371   
1  AAACAGCCAAGCGATG  ENCSR139ATR  GTCATAGACCCGCTGT          10459   

   atac_fragment_count  rna_frac_mit

In [12]:
# df_cell_types_for_atac_dataset[df_cell_types_for_atac_dataset['cell_id'] == 'GAGCGGTCAGCAATAA_ENCSR618WVK']


In [13]:
# cell_id_to_type_dict['ACTTAGTCAGATAGAC_ENCSR618WVK']

In [14]:
# cell_id_to_type_dict['ACAGCGGGTTTCCTCA_ENCSR618WVK']

In [15]:
# cell_id_to_type_dict['CTAGTGAGTACAATGT_ENCSR618WVK']

In [16]:
# cell_id_to_type_dict

In [17]:
# cell_id_to_type_dict
# {'ACAGCGGGTTTCCTCA_ENCSR618WVK': 'Hepatocytes',
#  'ACAGCGGGTCTATGAC_ENCSR618WVK': 'Hepatocytes',
#  'ACAGCGGGTGCGTAAA_ENCSR618WVK': 'Hepatocytes',
#  'ACAGCGGGTTAATCCA_ENCSR618WVK': nan,
#  'ACAGCGGGTTTACCAA_ENCSR618WVK': nan,
#  'ACAGCGGGTGCCGGAA_ENCSR618WVK': 'Hepatocytes',
#  'CATTTAGGTCAGTGCC_ENCSR618WVK': nan,
#  'CATTTAGGTCTTGAAC_ENCSR618WVK': 'Hepatocytes',
#  'CATTTAGGTGCACGCT_ENCSR618WVK': nan,
#  'CATTTAGGTAGCTTTA_ENCSR618WVK': 'Hepatocytes',
#  'CATTTAGGTCATCATA_ENCSR618WVK': nan,
#  'CATTTAGGTACTTCGG_ENCSR618WVK': nan,
#  'CTTTATCGTTAAGTGG_ENCSR618WVK': nan,

In [18]:
# cell_id_to_type_dict

In [19]:
# cell_id_to_type_dict['ACCGAAGCATGAGCAG_ENCSR618WVK']

In [20]:
# cell_id_to_type_dict['TGAGGAAACCCGCTGT_ENCSR618WVK']

In [21]:
# dict((key, value) for key, value in zip(df_cell_types_for_atac_dataset['cell_id'], df_cell_types_for_atac_dataset['cell_type_id']) if str(key)!="nan")


In [22]:
# !pwd

In [23]:
# def write_dict_to_tsv(dictionary, file_path):
#     with open(file_path, 'w') as tsvfile:
#         tsvfile.write("Key\tValue\n")
#         for key, value in dictionary.items():
#             tsvfile.write(f"{key}\t{value}\n")




In [24]:
# len(cell_id_to_type_dict)

In [25]:
# import numpy as np
# cleaned_dict = {key: value for key, value in cleaned_dict.items() if not isinstance(value, float) or not np.isnan(value)}


In [26]:
# len(cleaned_dict)